In [ ]:
#pip install scikit-learn pandas seaborn

In [ ]:
import sklearn as sk
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# Part A. Linear Regression From Scratch

Creating dataframes for the targets and features

In [ ]:
X = pd.DataFrame(fetch_california_housing()['data'], columns=fetch_california_housing()['feature_names'])
y = pd.DataFrame(fetch_california_housing()['target'], columns=fetch_california_housing()['target_names'])

Reviewing the data

In [ ]:
display(X.info())
display(y.info())
print('There are no missings')

In [ ]:
display(X.describe())
display(y.describe())

In [ ]:
plt.hist(y['MedHouseVal'], bins=20, edgecolor='black', alpha=0.7) 
plt.title('MedHouseVal')
plt.show
print('There are not ouliers')

In [ ]:
for target in ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']:
    plt.hist(X[target], bins=15, edgecolor='black', alpha=0.7)
    plt.title(target)
    plt.show()

In [ ]:
for target in ['AveRooms', 'AveBedrms', 'Population', 'AveOccup']:
    plt.boxplot(X[target])
    plt.title(target)
    plt.show()

In [ ]:
for target in ['AveRooms', 'AveBedrms', 'Population', 'AveOccup']:
    fig, axs = plt.subplots(1, 3, figsize=(15, 4)) 
    axs[0].hist(X[target], bins=15, edgecolor='black', alpha=0.7)
    axs[0].set_title(target)
    axs[1].hist(np.log(X[target]), bins=15, edgecolor='black', alpha=0.7)
    axs[1].set_title(target)
    axs[2].boxplot(np.log(X[target]))
    axs[2].set_title(target)
    plt.show()

In [ ]:
df = X.copy()

o_rows = len(df)

print(o_rows)

for feature in ['AveRooms', 'AveBedrms', 'Population', 'AveOccup']:
    q1 = np.percentile(np.log(df[feature]), 25)
    q3 = np.percentile(np.log(df[feature]), 75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    df = df[(np.log(df[feature])<upper_bound) & (np.log(df[feature])>lower_bound)]

    print(len(df))

#    fig, axs = plt.subplots(1, 3, figsize=(15, 4)) 
#    axs[0].hist(X[target],

m_rows = len(df)

print(m_rows)

print(f'The dataset ends with {np.round(m_rows/o_rows,2)}% of the original rows')

X_mod = df
y_mod = X_mod.join(y, how='inner', lsuffix='_del')[['MedHouseVal']]

Vizualiting correlations

In [ ]:
df_combined = pd.concat([X_mod, y_mod], axis=1)


corr_matrix = df_combined.corr()
target_col = y_mod.columns[0]
corr_with_target = corr_matrix[[target_col]].drop(index=target_col)
corr_features = X_mod.corr()


plt.figure(figsize=(8, len(corr_with_target) * 0.5))
sns.heatmap(corr_with_target, annot=True, cmap='coolwarm', center=0)
plt.title(f'Correlation features with {target_col}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 6))
sns.heatmap(corr_features, annot=True, cmap='mako', center=0)
plt.title(f'Correlation features')
plt.tight_layout()
plt.show()

print('Longitude, Population and AveBedrms are discarted for the poor correlation with the target. Surprisingly Latitude have a minimun correlation whit MedHouseVal')

X_mod_2 = X_mod[['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude']]

Standardizing features

In [ ]:
X_std = X_mod_2.copy()
for feature in X_std.columns:
    X_std[feature] = (X_std[feature] - np.mean(X_std[feature])) / np.std(X_std[feature])


Spliting the dataset in train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y_mod, test_size=0.2, random_state=1234)
print('We are ready!')

Closed-form OLS

In [ ]:
X_train_wi = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
beta_hat = np.linalg.inv(X_train_wi.T @ X_train_wi) @ X_train_wi.T @ y_train

In [ ]:
for i, f in enumerate(['Intercept'] + [i for i in X_train.columns]):
    print(f'Coefficient of {f}: {beta_hat.iloc[i,:][0]}')

In [ ]:
y_train_pred = X_train_wi @ beta_hat

In [ ]:
X_test_wi = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

In [ ]:
y_test_pred = X_test_wi @ beta_hat

In [ ]:
e = y_test_pred - y_test.reset_index(drop=True)

In [ ]:
x = range(len(y_test_pred))
plt.figure(figsize=(20, 5))
plt.plot(e, label='Valor real', color='blue')
#plt.plot(x, y_test, label='Valor real', color='red')
plt.ylabel('Error')
plt.title('Errors in test')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(y_test, y_test_pred, alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--', lw=2)

plt.xlabel("Real")
plt.ylabel("Predict")
plt.title("Real values vs. predicted values in test")
plt.grid(True)
plt.show()

Now with gradient descent

In [ ]:
def f(X, beta):
    return X @ beta
def mse_gradient(beta, X, y):
    return np.mean((f(X, beta) - y) * X.T, axis=1)
def mean_squared_error(beta, X, y):
    return np.mean((y-f(X, beta))**2)

In [ ]:
threshold = 1e-3
step_size = 4e-1
beta, beta_prev = np.array([5,4,3,2,1]), np.ones(5,)
opt_pts = [beta]
opt_grads = []
iter = 0

while np.linalg.norm(beta - beta_prev) > threshold:
    if iter % 100 == 0:
        print('Iteration %d. MSE: %.6f' % (iter, mean_squared_error(beta, X_train, y_train)))
    beta_prev = beta
    gradient = mse_gradient(beta, X_train, y_train)
    beta = beta_prev - step_size * gradient
    opt_pts += [beta]
    opt_grads += [gradient]
    iter += 1

In [ ]:
np.ones(2,)